**Эксперимент с отдельной векторизацией каждого столба и использованием PCA**

In [459]:
import json
import os
import sys

import joblib

# Добавляем корневую директорию проекта в sys.path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "../../")))

# Импортируем metrics из utils
from utils import metrics as mtc
from utils import preprocess_functions as ppf

In [460]:
import numpy as np
import pandas as pd
import random

# Готовим дата-сеты (предобработка)

In [461]:
reference = pd.read_csv("../../data/preprocessed/general/reference.csv")
train = pd.read_csv("../../data/splited/train.csv")
valid = pd.read_csv("../../data/splited/valid.csv")

abbr_dict = joblib.load("../../resources/general/abbreviations_dict.joblib")
region_dict = joblib.load("../../resources/general/region_dict.joblib")
blacklist_opf = joblib.load("../../resources/general/blacklist_opf.joblib")


In [462]:
reference["processed_name"] = reference.name.apply(
    ppf.simple_preprocess_text
).str.lower()
reference.processed_name = reference.processed_name.apply(ppf.replace_numbers_with_text)
reference.processed_name = reference.processed_name.apply(
    ppf.abbr_preprocess_text, args=(abbr_dict, False, False, True, False)
)
reference.processed_name = reference.processed_name.apply(
    ppf.process_region, args=(list(region_dict.keys()),)
)
reference.processed_name = reference.processed_name.apply(
    ppf.remove_substrings, args=(blacklist_opf,)
)
reference.head(1)


,id,name,region,processed_name
0,69,Кировска,мурманская область,кировска


In [463]:
train["processed_name"] = train.name.apply(ppf.simple_preprocess_text).str.lower()
train.processed_name = train.processed_name.apply(ppf.replace_numbers_with_text)
train.processed_name = train.processed_name.apply(
    ppf.abbr_preprocess_text, args=(abbr_dict, False, False, True, False)
)
train["region"] = train.processed_name.apply(
    ppf.process_region, args=(list(region_dict.keys()), True)
)

train.processed_name = train.processed_name.apply(
    ppf.process_region, args=(list(region_dict.keys()),)
)
train.processed_name = train.processed_name.apply(
    ppf.remove_substrings, args=(blacklist_opf,)
)
train.head(1)


,name,school_id,processed_name,region
0,"ХМАО-Югра, СШ г. Покачи",173,спортивная школа покачи,ханты мансийский автономный округ


In [464]:
valid["processed_name"] = valid.name.apply(ppf.simple_preprocess_text).str.lower()
valid.processed_name = valid.processed_name.apply(ppf.replace_numbers_with_text)
valid.processed_name = valid.processed_name.apply(
    ppf.abbr_preprocess_text, args=(abbr_dict, False, False, True, False)
)
valid["region"] = valid.processed_name.apply(
    ppf.process_region, args=(list(region_dict.keys()), True)
)

valid.processed_name = valid.processed_name.apply(
    ppf.process_region, args=(list(region_dict.keys()),)
)
valid.processed_name = valid.processed_name.apply(
    ppf.remove_substrings, args=(blacklist_opf,)
)
valid.head(1)


,name,school_id,processed_name,region
0,КФК Е. Герасимовой,1843,клуб фигурного катания герасимовой,None


# Данные для обучения

In [465]:
merged_train = train.join(
    reference.set_index("id"), on="school_id", how="left", rsuffix="_reference"
)
merged_train = merged_train[~merged_train.name_reference.isna()]
merged_train = merged_train[~merged_train.processed_name.isna()]
merged_train = merged_train[
    [
        "processed_name",
        "region",
        "processed_name_reference",
        "region_reference",
    ]
]
merged_train["target"] = 1

In [466]:
merged_train.head()

,processed_name,region,processed_name_reference,region_reference,target
0,спортивная школа покачи,ханты мансийский автономный округ,спортивная школа,ханты мансийский автономный округ,1
1,спортивная школа олимпийского резерва по фигур...,санкт петербург,спортивная школа олимпийского резерва академия...,санкт петербург,1
2,спортивная школа ямал,ямало ненецкий автономный округ,ямал,ямало ненецкий автономный округ,1
3,смольнинское секция фигурного катания ice drea...,None,ice dream айс дрим,санкт петербург,1
4,московская академия фигурного катания на коньк...,москва,олимп,москва,1


In [467]:
temp_df = merged_train.copy()
train_classif = merged_train.copy()

for i in range(0, 5, 1):
    temp_df = temp_df.copy()

    # Параметр для сдвига
    shift_amount = random.randint(0, 25)
    shift_amount = 15

    # Сдвиг второго столбца
    temp_df["processed_name_reference"] = temp_df["processed_name_reference"].shift(
        shift_amount
    )
    temp_df["region_reference"] = temp_df["region_reference"].shift(shift_amount)
    temp_df["target"] = 0

    # Новый датафрейм с исходным и сдвинутым столбцами
    temp_df = temp_df.dropna()

    train_classif = pd.concat([train_classif, temp_df])

train_classif.reset_index(drop=True, inplace=True)

In [468]:
train_classif.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2433 entries, 0 to 2432
Data columns (total 5 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   processed_name            2433 non-null   object
 1   region                    2310 non-null   object
 2   processed_name_reference  2433 non-null   object
 3   region_reference          2433 non-null   object
 4   target                    2433 non-null   int64 
dtypes: int64(1), object(4)
memory usage: 95.2+ KB


In [469]:
train_classif = train_classif.fillna(" ")

In [470]:
# Объединение текстов для векторизации
train_classif["combined"] = (
    (
        train_classif["processed_name"]
        + " "
        + train_classif["region"]
        + " "
        + train_classif["processed_name_reference"]
        + " "
        + train_classif["region_reference"]
        + " "
    )
    .str.strip()
    .str.replace("  ", " ")
)

In [471]:
# Разделение на признаки и целевую переменную
X = train_classif["combined"]
y = train_classif["target"]

In [472]:
from sklearn.model_selection import train_test_split

# Разделение данных на обучающую и тестовую выборки

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

# CATBOOST

In [473]:
from catboost import CatBoostClassifier
from imblearn.pipeline import Pipeline as ImbPipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report
from imblearn.over_sampling import SMOTE

In [474]:
# Создание пайплайна для векторизации, балансировки классов и обучения модели логистической регрессии
pipeline = ImbPipeline(
    [
        ("tfidf", TfidfVectorizer()),
        ("smote", SMOTE(sampling_strategy="auto", k_neighbors=5, random_state=42)),
        (
            "clf",
            CatBoostClassifier(
                iterations=100, verbose=100, auto_class_weights="Balanced"
            ),
        ),
    ]
)

In [475]:
# Обучение модели
pipeline.fit(X_train, y_train)

Learning rate set to 0.136503
0:	learn: 0.5844948	total: 13.4ms	remaining: 1.32s


99:	learn: 0.1022499	total: 1.41s	remaining: 0us


Pipeline(steps=[('tfidf', TfidfVectorizer()), ('smote', SMOTE(random_state=42)),
                ('clf',
                 <catboost.core.CatBoostClassifier object at 0x00000204F1A9BF20>)])

In [476]:
# Оценка модели
y_pred_test = pipeline.predict(X_test)

In [478]:
print(classification_report(y_test, y_pred_test))

              precision    recall  f1-score   support

           0       0.99      0.93      0.96       378
           1       0.80      0.97      0.88       109

    accuracy                           0.94       487
   macro avg       0.90      0.95      0.92       487
weighted avg       0.95      0.94      0.94       487



# Данные для валидации

In [479]:
merged_valid = train.join(
    reference.set_index("id"), on="school_id", how="left", rsuffix="_reference"
)
merged_valid = merged_valid[~merged_valid.name_reference.isna()]
merged_valid = merged_valid[~merged_valid.processed_name.isna()]
merged_valid = merged_valid[
    [
        "processed_name",
        "region",
        "processed_name_reference",
        "region_reference",
    ]
]

merged_valid = merged_valid.fillna(" ")

In [480]:
merged_valid.head()

,processed_name,region,processed_name_reference,region_reference
0,спортивная школа покачи,ханты мансийский автономный округ,спортивная школа,ханты мансийский автономный округ
1,спортивная школа олимпийского резерва по фигур...,санкт петербург,спортивная школа олимпийского резерва академия...,санкт петербург
2,спортивная школа ямал,ямало ненецкий автономный округ,ямал,ямало ненецкий автономный округ
3,смольнинское секция фигурного катания ice drea...,,ice dream айс дрим,санкт петербург
4,московская академия фигурного катания на коньк...,москва,олимп,москва


In [481]:
valid_pos_classif = pd.DataFrame()

# Объединение текстов для векторизации
valid_pos_classif["left"] = merged_valid["processed_name"]
valid_pos_classif["left_region"] = merged_valid["region"]
valid_pos_classif["right"] = merged_valid["processed_name_reference"]
valid_pos_classif["right_region"] = merged_valid["region_reference"]

valid_pos_classif = valid_pos_classif.fillna(" ")

valid_pos_classif["combined"] = (
    valid_pos_classif["left"]
    + " "
    + valid_pos_classif["left_region"]
    + " "
    + valid_pos_classif["right"]
    + " "
    + valid_pos_classif["right_region"]
)
valid_pos_classif = valid_pos_classif.reset_index(drop=True)

In [482]:
# Оценка модели
y_pred_valid_pos = pipeline.predict(valid_pos_classif["combined"])

In [483]:
y_pred_valid_pos

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [484]:
# Качество предсказания класса 1
sum(y_pred_valid_pos) / len(y_pred_valid_pos)

0.990791896869245

In [485]:
valid_neg_classif = pd.DataFrame()

# Объединение текстов для векторизации
valid_neg_classif["left"] = merged_valid["processed_name"]
valid_neg_classif["left_region"] = merged_valid["region"]
valid_neg_classif["right"] = merged_valid["processed_name_reference"].shift(15)
valid_neg_classif["right_region"] = merged_valid["region_reference"].shift(15)

valid_neg_classif["combined"] = (
    valid_neg_classif["left"]
    + " "
    + valid_neg_classif["left_region"]
    + " "
    + valid_neg_classif["right"]
    + " "
    + valid_neg_classif["right_region"]
)

valid_neg_classif = valid_neg_classif[~valid_neg_classif.combined.isna()]

valid_neg_classif = valid_neg_classif.reset_index(drop=True)

In [487]:
# Оценка модели
y_pred_valid_neg = pipeline.predict(valid_neg_classif["combined"])

In [488]:
y_pred_valid_neg

array([1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0,
       1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1,
       0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [489]:
# Качество предсказания класса 0
(len(y_pred_valid_neg) - sum(y_pred_valid_neg)) / len(y_pred_valid_neg)

0.7973484848484849

# Изучение остатков

In [490]:
ids_pos_wrong = np.where(y_pred_valid_pos == 0)[0]
len(ids_pos_wrong.tolist())

5

In [491]:
ids_pos_wrong

array([260, 325, 437, 443, 477], dtype=int64)

In [493]:
valid_pos_classif.loc[
    ids_pos_wrong.tolist(),
    [
        "left",
        "left_region",
        "right",
        "right_region",
    ],
]

,left,left_region,right,right_region
260,звездочка,,звездочка,архангельская область
325,спортивная школа олимпийского резерва,санкт петербург,спортивная школа олимпийского резерва академия...,санкт петербург
437,сам область московская академия фигурного ката...,москва,вдохновение,москва
443,челябинская область спортивная школа олимпийск...,краснодарский край,спортивная школа олимпийского резерва по фигур...,краснодарский край
477,всероссийское физкультурно спортивное общество...,владимирская область,динамо,владимирская область


In [494]:
ids_neg_wrong = np.where(y_pred_valid_neg == 1)[0]
len(ids_neg_wrong.tolist())

107

In [495]:
valid_neg_classif.loc[ids_neg_wrong.tolist(), ["left", "right"]].head(10)

,left,right
0,школа фигурного катания маска,спортивная школа
1,спортивный клуб юнармецны,спортивная школа олимпийского резерва академия...
2,центр физической культуры спорта и здоровья мо...,ямал
3,спортивная школа олимпийского резерва звездный...,ice dream айс дрим
4,светлановское ледовая история,олимп
9,династия,спортивная школа олимпийского резерва по фигур...
15,гавань центр физической культуры спорта и здор...,маска
16,смольнинский секция фигурного катания ice drea...,спортивный клуб юнармецны
22,локомотив,детско юношеская спортивная школа десять
24,физкультурно спортивное объединение локомотив,династия


# Применение модели

In [496]:
reference_id = reference["id"].to_numpy(dtype="int").flatten()
reference_name = reference["processed_name"].to_numpy(dtype="str").flatten()
reference_region = reference["region"].to_numpy(dtype="str").flatten()

In [497]:
joblib.dump(reference_id, "../../resources/exp43/reference_id.joblib")
joblib.dump(reference_name, "../../resources/exp43/reference_name.joblib")
joblib.dump(reference_region, "../../resources/exp43/reference_region.joblib")

['../../resources/exp43/reference_region.joblib']

In [498]:
x_valid = valid["processed_name"].to_numpy(dtype="str").flatten()
y_valid = valid["school_id"].to_numpy(dtype="int").flatten()
region_valid = valid["region"].to_numpy(dtype="str").flatten()

In [499]:
x_valid_process = np.array([f"{xi} {yi}" for xi, yi in zip(x_valid, region_valid)])
reference_process = np.array(
    [f"{xi} {yi}" for xi, yi in zip(reference_name, reference_region)]
)

In [508]:
y_pred_final = []

for i, x_process in enumerate(x_valid_process):
    reference_ids = np.where(reference_region == region_valid[i])[0]
    print(reference_ids)
    if len(reference_ids) != 0:
        x_test_process = np.char.add(x_process + " ", reference_process[reference_ids])
        print(x_test_process)
        print()
    else:
        x_test_process = np.char.add(x_process + " ", reference_process)

    # Оценка модели
    y_pred_test = pipeline.predict_proba(x_test_process)[:, 1]

    # Найти индексы, отсортированные по убыванию значений
    sorted_indices = np.argsort(y_pred_test)[::-1]

    # Выбрать первые пять индексов
    top_five_indices = sorted_indices[:5]

    # Получить значения этих элементов
    top_five_values = y_pred_test[top_five_indices]

    # Создать массив с кортежами (id элемента, значение)
    result = np.array(
        [
            (reference_id[index], value)
            for index, value in zip(top_five_indices, top_five_values)
        ],
        dtype=[("index", int), ("value", float)],
    )

    y_pred_final.append(result)

[]
[  6  17  22  26  32  36  62  80  81  92  96  97  99 100 106 111 126 155
 161 191 206 207 217 226 273 274 281 288 295 296 297]
['московская академия фигурного катания на коньках школа хрустальный москва конек чайковской москва'
 'московская академия фигурного катания на коньках школа хрустальный москва академия синхронного катания на коньках москва'
 'московская академия фигурного катания на коньках школа хрустальный москва ангелы плющенко москва'
 'московская академия фигурного катания на коньках школа хрустальный москва армия фигурного катания москва'
 'московская академия фигурного катания на коньках школа хрустальный москва вдохновение москва'
 'московская академия фигурного катания на коньках школа хрустальный москва волкова москва'
 'московская академия фигурного катания на коньках школа хрустальный москва прусова москва'
 'московская академия фигурного катания на коньках школа хрустальный москва созвездие москва'
 'московская академия фигурного катания на коньках школа хруста

In [509]:
y_pred_final

[array([(164, 0.97774113), ( 62, 0.97418175), (264, 0.97328042),
        (  8, 0.9713713 ), ( 90, 0.96853809)],
       dtype=[('index', '<i4'), ('value', '<f8')]),
 array([(10, 0.94840413), (18, 0.91484799), ( 8, 0.91484799),
        (25, 0.9097432 ), (70, 0.88513543)],
       dtype=[('index', '<i4'), ('value', '<f8')]),
 array([(282, 0.99053018), ( 63, 0.97528212), (272, 0.96153933),
        (286, 0.96153933), (105, 0.96011462)],
       dtype=[('index', '<i4'), ('value', '<f8')]),
 array([(69, 0.86763212), (70, 0.64206852), ( 8, 0.5872569 ),
        (71, 0.48481252)], dtype=[('index', '<i4'), ('value', '<f8')]),
 array([( 66, 0.89474694), (  9, 0.89474694), ( 22, 0.89474694),
        (149, 0.89474694), ( 84, 0.89474694)],
       dtype=[('index', '<i4'), ('value', '<f8')]),
 array([(77, 0.9521467 ), ( 8, 0.84507988), (72, 0.73934073),
        (80, 0.67586438), (73, 0.65825106)],
       dtype=[('index', '<i4'), ('value', '<f8')]),
 array([(10, 0.96640102), (25, 0.93997356), ( 8, 0.93188

In [502]:
x_valid_process = np.array([f"{xi} {yi}" for xi, yi in zip(x_valid, region_valid)])
reference_process = np.array(
    [f"{xi} {yi}" for xi, yi in zip(reference_name, reference_region)]
)

In [503]:
y_pred_final = []

for x_process in x_valid_process:
    x_test_process = np.char.add(x_process + " ", reference_process)
    # print(x_test_process)
    # Оценка модели
    y_pred_test = pipeline.predict_proba(x_test_process)[:, 1]

    # Найти индексы, отсортированные по убыванию значений
    sorted_indices = np.argsort(y_pred_test)[::-1]

    # Выбрать первые пять индексов
    top_five_indices = sorted_indices[:5]

    # Получить значения этих элементов
    top_five_values = y_pred_test[top_five_indices]

    # Создать массив с кортежами (id элемента, значение)
    result = np.array(
        [
            (reference_id[index], value)
            for index, value in zip(top_five_indices, top_five_values)
        ],
        dtype=[("index", int), ("value", float)],
    )

    y_pred_final.append(result)

# Преобразовать итоговый список в массив numpy
y_pred_final = np.array(y_pred_final)

In [504]:
y_pred_final

array([[( 164, 0.97774113), (  62, 0.97418175), ( 264, 0.97328042),
        (   8, 0.9713713 ), (  90, 0.96853809)],
       [( 244, 0.94840413), ( 293, 0.91484799), (  20, 0.91484799),
        (1845, 0.9097432 ), (  10, 0.88513543)],
       [( 282, 0.99053018), (  63, 0.97528212), ( 272, 0.96153933),
        ( 286, 0.96153933), ( 105, 0.96011462)],
       [(   7, 0.86763212), ( 124, 0.64206852), ( 300, 0.5872569 ),
        ( 282, 0.55515934), ( 610, 0.52404723)],
       [(  66, 0.89474694), (   9, 0.89474694), (  22, 0.89474694),
        ( 149, 0.89474694), (  84, 0.89474694)],
       [( 110, 0.9521467 ), ( 282, 0.9295889 ), ( 107, 0.9083839 ),
        ( 141, 0.85371002), (  21, 0.84507988)],
       [( 244, 0.96640102), (1845, 0.93997356), ( 293, 0.9318869 ),
        (  20, 0.9318869 ), ( 282, 0.90848216)],
       [(  86, 0.91694407), ( 282, 0.86786426), ( 117, 0.72387741),
        ( 300, 0.58821579), ( 141, 0.58390125)],
       [( 282, 0.98334204), (  20, 0.94037624), (1845, 0.9403762

In [510]:
y_pred_final_1 = [k[0][0] for k in y_pred_final]

In [511]:
df = pd.DataFrame(
    [(y_valid[i], y_pred_final_1[i], y_pred_valid_pos[i]) for i in range(0, 181)],
    columns=["y_valid", "y_pred_final_1", "y_pred_valid_pos"],
)
df

,y_valid,y_pred_final_1,y_pred_valid_pos
0,1843,164,1
1,244,10,1
2,63,282,1
3,7,69,1
4,50,66,1
...,...,...,...
176,23,70,1
177,256,253,1
178,41,70,1
179,264,76,1


# Результат

In [512]:
metrics = mtc.calculate_metrics(y_valid, y_pred_final, [])
metrics

{'Accuracy@1': 0.022,
 'Accuracy@3': 0.083,
 'Accuracy@5': 0.105,
 'Accuracy@10': 0.105,
 'auto_error_rate': 0.978,
 'manual_processing_rate': 0.0,
 'general_error': 0.978}

In [186]:
# Сохраняем словарь в файл JSON
with open("../../evaluations/exp43.json", "w", encoding="utf-8") as f:
    json.dump(metrics, f, ensure_ascii=False, indent=4)

# Попытка работы с PCA

In [513]:
from sklearn.decomposition import PCA

In [514]:
train_classif.head()

,processed_name,region,processed_name_reference,region_reference,target,combined
0,спортивная школа покачи,ханты мансийский автономный округ,спортивная школа,ханты мансийский автономный округ,1,спортивная школа покачи ханты мансийский автон...
1,спортивная школа олимпийского резерва по фигур...,санкт петербург,спортивная школа олимпийского резерва академия...,санкт петербург,1,спортивная школа олимпийского резерва по фигур...
2,спортивная школа ямал,ямало ненецкий автономный округ,ямал,ямало ненецкий автономный округ,1,спортивная школа ямал ямало ненецкий автономны...
3,смольнинское секция фигурного катания ice drea...,,ice dream айс дрим,санкт петербург,1,смольнинское секция фигурного катания ice drea...
4,московская академия фигурного катания на коньк...,москва,олимп,москва,1,московская академия фигурного катания на коньк...


In [515]:
# Разделение на признаки и целевую переменную
# X_pca = train_classif.drop(["target", "combined"], axis=1)
X_pca = train_classif.drop(["target"], axis=1)
y_pca = train_classif["target"]


In [516]:
from sklearn.model_selection import train_test_split

# Разделение данных на обучающую и тестовую выборки

X_train_pca, X_test_pca, y_train_pca, y_test_pca = train_test_split(
    X_pca, y_pca, test_size=0.2, stratify=y_pca, random_state=42
)


In [517]:
# Создание общего массива строк для обучения TF-IDF векторизатора
all_names = pd.concat(
    [X_train_pca["processed_name"], X_train_pca["processed_name_reference"]]
)
all_regions = pd.concat([X_train_pca["region"], X_train_pca["region_reference"]])


In [518]:
# Обучение TF-IDF векторизатора
vectorizer_names = TfidfVectorizer().fit(all_names)
vectorizer_regions = TfidfVectorizer().fit(all_regions)


In [519]:
all_names_vec = vectorizer_names.transform(all_names)
all_regions_vec = vectorizer_regions.transform(all_regions)
left_vec = vectorizer_names.transform(X_train_pca["processed_name"])
left_region_vec = vectorizer_regions.transform(X_train_pca["region"])
right_vec = vectorizer_names.transform(X_train_pca["processed_name_reference"])
right_region_vec = vectorizer_regions.transform(X_train_pca["region_reference"])


In [520]:
# Обучение PCA на общем массиве строк
pca_names = PCA(n_components=3).fit(all_names_vec.toarray())
pca_regions = PCA(n_components=3).fit(all_regions_vec.toarray())


In [521]:
# Применение PCA к результату векторизации для каждого столбца
left_vec_pca = pca_names.transform(left_vec.toarray())
left_region_vec_pca = pca_regions.transform(left_region_vec.toarray())
right_vec_pca = pca_names.transform(right_vec.toarray())
right_region_vec_pca = pca_regions.transform(right_region_vec.toarray())


In [522]:
# Объединение результатов PCA для обоих столбцов
X_train_pca_combined = np.hstack(
    (
        left_vec_pca,
        # left_region_vec_pca,
        right_vec_pca,
        # right_region_vec_pca,
    )
)


In [523]:
# Создание дата-фрейма из результатов PCA
df_pca = pd.DataFrame(X_train_pca_combined).add_prefix("pca_")


# CATBOOST

In [524]:
from catboost import CatBoostClassifier
from imblearn.pipeline import Pipeline as ImbPipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report
from imblearn.over_sampling import SMOTE


In [525]:
# Создание пайплайна для векторизации, балансировки классов и обучения модели логистической регрессии
pipeline_pca = ImbPipeline(
    [
        ("smote", SMOTE(sampling_strategy="auto", k_neighbors=5, random_state=42)),
        (
            "clf",
            CatBoostClassifier(
                iterations=1000, verbose=100, auto_class_weights="SqrtBalanced"
            ),
        ),
    ]
)


In [526]:
# Обучение модели
pipeline_pca.fit(X_train_pca_combined, y_train_pca)


Learning rate set to 0.016525
0:	learn: 0.6852798	total: 4.05ms	remaining: 4.05s
100:	learn: 0.3654534	total: 369ms	remaining: 3.29s
200:	learn: 0.2861857	total: 680ms	remaining: 2.7s
300:	learn: 0.2486483	total: 1.01s	remaining: 2.34s
400:	learn: 0.2227885	total: 1.33s	remaining: 1.98s
500:	learn: 0.2019156	total: 1.63s	remaining: 1.63s
600:	learn: 0.1855680	total: 1.93s	remaining: 1.28s
700:	learn: 0.1724363	total: 2.23s	remaining: 951ms
800:	learn: 0.1601801	total: 2.52s	remaining: 627ms
900:	learn: 0.1484920	total: 2.81s	remaining: 309ms
999:	learn: 0.1392520	total: 3.11s	remaining: 0us


Pipeline(steps=[('smote', SMOTE(random_state=42)),
                ('clf',
                 <catboost.core.CatBoostClassifier object at 0x0000020480E6C440>)])

In [527]:
left_vec_test = vectorizer_names.transform(X_test_pca["processed_name"])
left_region_vec_test = vectorizer_regions.transform(X_test_pca["region"])
right_vec_test = vectorizer_names.transform(X_test_pca["processed_name_reference"])
right_region_vec_test = vectorizer_regions.transform(X_test_pca["region_reference"])


In [528]:
# Применение PCA к результату векторизации для каждого столбца
left_vec_pca_test = pca_names.transform(left_vec_test.toarray())
left_region_vec_pca_test = pca_regions.transform(left_region_vec_test.toarray())
right_vec_pca_test = pca_names.transform(right_vec_test.toarray())
right_region_vec_pca_test = pca_regions.transform(right_region_vec_test.toarray())


In [529]:
# Объединение результатов PCA для обоих столбцов
X_test_pca_combined = np.hstack(
    (
        left_vec_pca_test,
        # left_region_vec_pca_test,
        right_vec_pca_test,
        # right_region_vec_pca_test,
    )
)


In [530]:
# Оценка модели
y_pred_test_pca_proba = pipeline_pca.predict_proba(X_test_pca_combined)[:, 1]
y_pred_test_pca = pipeline_pca.predict(X_test_pca_combined)


In [531]:
print(classification_report(y_test_pca, y_pred_test_pca))

              precision    recall  f1-score   support

           0       0.96      0.86      0.91       378
           1       0.65      0.88      0.75       109

    accuracy                           0.87       487
   macro avg       0.81      0.87      0.83       487
weighted avg       0.89      0.87      0.87       487



In [532]:
true_id = np.where(y_test_pca == 1)[0]

In [533]:
residuals_df_pos = X_test_pca.iloc[true_id].copy()

In [534]:
residuals_df_pos["y_pred"] = y_pred_test_pca_proba[true_id]

In [535]:
residuals_df_pos[residuals_df_pos.y_pred < 0.5]

,processed_name,region,processed_name_reference,region_reference,combined,y_pred
106,республиканская спортивная школа олимпийского ...,республика татарстан,республиканская спортивная школа олимпийского ...,республика татарстан,республиканская спортивная школа олимпийского ...,0.005615
107,академия фигурного катания наши надежды,московская область,наши надежды,московская область,академия фигурного катания наши надежды москов...,0.258777
162,спортивная школа дворец спорта северная звезда,нижегородская область,северная звезда,нижегородская область,спортивная школа дворец спорта северная звезда...,0.047026
406,спортивная школа по фигурного катания на коньк...,брянская область,брянск,брянская область,спортивная школа по фигурного катания на коньк...,0.263172
506,ледовый спортивный комплекс лыткарино,московская область,лыткарино,московская область,ледовый спортивный комплекс лыткарино московск...,0.291778
331,детско юношеская спортивная школа невский лед,санкт петербург,невский лед,санкт петербург,детско юношеская спортивная школа невский лед ...,0.331291
444,маффк мечта,москва,мечта,москва,маффк мечта москва мечта москва,0.435094
477,всероссийское физкультурно спортивное общество...,владимирская область,динамо,владимирская область,всероссийское физкультурно спортивное общество...,0.498262
325,спортивная школа олимпийского резерва,санкт петербург,спортивная школа олимпийского резерва академия...,санкт петербург,спортивная школа олимпийского резерва санкт пе...,0.061543
294,спортивная школа одинцово училище техникум оли...,московская область,одинцово,московская область,спортивная школа одинцово училище техникум оли...,0.074573


In [536]:
false_id = np.where(y_test_pca == 0)[0]

In [537]:
residuals_df_neg = X_test_pca.iloc[false_id].copy()

In [538]:
residuals_df_neg["y_pred"] = y_pred_test_pca_proba[false_id]

In [539]:
residuals_df_neg[residuals_df_neg.y_pred > 0.5]

,processed_name,region,processed_name_reference,region_reference,combined,y_pred
1439,ароо федерация фигурного катания созвездие,астраханская область,мастер,санкт петербург,ароо федерация фигурного катания созвездие аст...,0.565224
1897,федерация фигурного катания на коньках ставроп...,ставропольский край,олимп,калужская область,федерация фигурного катания на коньках ставроп...,0.620068
2329,федерация фигурного катания,республика башкортостан,школа фигурного катания бережной,санкт петербург,федерация фигурного катания республика башкорт...,0.821003
1190,спортивный клуб фигурного катания северное сияние,санкт петербург,созвездие,москва,спортивный клуб фигурного катания северное сия...,0.702834
922,спортивная школа,кемеровская область,спортивная школа олимпийского резерва академия...,санкт петербург,спортивная школа кемеровская область спортивна...,0.925154
1538,российская федерация центральный спортивный кл...,москва,спортивный клуб фигурного катания москвиной,санкт петербург,российская федерация центральный спортивный кл...,0.545982
1977,спортивная школа холмск арена,сахалинская область,юность,свердловская область,спортивная школа холмск арена сахалинская обла...,0.619099
1334,первоуральск первоуральское физической культу...,свердловская область,старт,пермский край,первоуральск первоуральское физической культур...,0.795709
1503,санкт петербрг спортивная школа олимпийского...,санкт петербург,юность,свердловская область,санкт петербрг спортивная школа олимпийского ...,0.576902
1994,спортивная школа олимпийского резерва старт,пермский край,спортивная школа олимпийского резерва тридцать...,омская область,спортивная школа олимпийского резерва старт пе...,0.530744


# Тест с учетом региона

In [407]:
X_test_pca_sim_region = X_test_pca[X_test_pca.region == X_test_pca.region_reference]
X_test_pca_sim_region.info()


<class 'pandas.core.frame.DataFrame'>
Index: 120 entries, 374 to 2476
Data columns (total 4 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   processed_name            120 non-null    object
 1   region                    120 non-null    object
 2   processed_name_reference  120 non-null    object
 3   region_reference          120 non-null    object
dtypes: object(4)
memory usage: 4.7+ KB


In [414]:
X_test_pca_sim_region.index

Index([ 374,   99,  791,  419,  250,  297,  265,  525,  281,  420,
       ...
       1940,  716,  529,  942,  952,  511,  304,   79, 1702, 2476],
      dtype='int64', length=120)

In [409]:
left_vec_test_sim_region = vectorizer_names.transform(
    X_test_pca_sim_region["processed_name"]
)
right_vec_test_sim_region = vectorizer_names.transform(
    X_test_pca_sim_region["processed_name_reference"]
)


In [410]:
# Применение PCA к результату векторизации для каждого столбца
left_vec_pca_test_sim_region = pca_names.transform(left_vec_test.toarray())
right_vec_pca_test_sim_region = pca_names.transform(right_vec_test.toarray())


In [411]:
# Объединение результатов PCA для обоих столбцов
X_test_pca_combined_sim_region = np.hstack(
    (
        left_vec_pca_test_sim_region,
        right_vec_pca_test_sim_region,
    )
)


In [412]:
# Оценка модели
y_pred_test_pca_proba_sim_region = pipeline_pca.predict_proba(
    X_test_pca_combined_sim_region
)[:, 1]
y_pred_test_pca_sim_region = pipeline_pca.predict(X_test_pca_combined_sim_region)


In [416]:
print(
    classification_report(
        y_test_pca.loc[X_test_pca_sim_region.index], y_pred_test_pca_sim_region
    )
)


              precision    recall  f1-score   support

           0       0.69      0.75      0.72        32
           1       0.91      0.88      0.89        88

    accuracy                           0.84       120
   macro avg       0.80      0.81      0.80       120
weighted avg       0.85      0.84      0.84       120



In [429]:
true_id_sim_region = np.where(y_test_pca.loc[X_test_pca_sim_region.index] == 1)[0]

In [430]:
residuals_df_pos_sim_region = X_test_pca_sim_region.iloc[true_id_sim_region].copy()

In [431]:
residuals_df_pos_sim_region["y_pred"] = y_pred_test_pca_proba_sim_region[
    true_id_sim_region
]


In [432]:
residuals_df_pos_sim_region[residuals_df_pos_sim_region.y_pred < 0.5]

,processed_name,region,processed_name_reference,region_reference,y_pred
525,екатеринбург спортивная школа олимпийского р...,свердловская область,юность,свердловская область,0.412482
19,московская академия фигурного катания на коньк...,москва,олимп,москва,0.389371
457,клуб фигурного катания движение,республика татарстан,движение,республика татарстан,0.492067
197,спортивная школа прибой,тюменская область,прибой,тюменская область,0.450487
13,спортивная школа спартак,чувашская республика,спартак,чувашская республика,0.045270
230,московская академия фигурного катания на коньках,москва,московская академия фигурного катания на коньках,москва,0.307945
331,физкультурно спортивное объединение клуб фигур...,санкт петербург,звезда,санкт петербург,0.304284
456,спортивная школа рассвет,красноярский край,рассвет,красноярский край,0.304668
529,московская академия фигурного катания на коньках,москва,московская академия фигурного катания на коньках,москва,0.307945
511,физкультурно оздоровительный комплекс мещерский,нижегородская область,мещерский,нижегородская область,0.481976


In [433]:
false_id_sim_region = np.where(y_test_pca.loc[X_test_pca_sim_region.index] == 0)[0]

In [434]:
residuals_df_neg_sim_region = X_test_pca_sim_region.iloc[false_id_sim_region].copy()

In [435]:
residuals_df_neg_sim_region["y_pred"] = y_pred_test_pca_proba_sim_region[
    false_id_sim_region
]


In [436]:
residuals_df_neg_sim_region[residuals_df_neg_sim_region.y_pred > 0.5]

,processed_name,region,processed_name_reference,region_reference,y_pred
791,физкультурно спортивное объединение клуб фигур...,санкт петербург,спартак,санкт петербург,0.513316
2211,оренбург спортивная школа олимпийского резер...,оренбургская область,спортивная школа олимпийского резерва один,оренбургская область,0.746834
702,спортивный клуб армия фигурного катания,москва,легенда,москва,0.660056
1940,российская федерация центральный спортивный кл...,москва,центральный спортивный клуб армии,москва,0.974192
716,московская академия фигурного катания на коньк...,москва,вдохновение,москва,0.834084
942,всеволжская спортивная школа олимпийского резерва,ленинградская область,всеволжская спортивная школа олимпийского резерва,ленинградская область,0.917369
952,спортивная школа восемь локомотив,свердловская область,спортивная школа им александра козицына,свердловская область,0.646742
2476,московская академия фигурного катания на коньк...,москва,хрустальный,москва,0.717440
